In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.testing import assert_frame_equal
p1 = Path.cwd() / 'back_data'

In [2]:
# 탐색적 데이터 분석(EDA, Exploratory Data Analysis)은 데이터의 특성을 요약하고 시각화
# 일상적이고 체계적인 방식의 메타데이터(metadata, 데이터에 관한 데이터)와 기술 통계량 수집 진행
# 데이터 딕셔너리는 메타데이터의 테이블, 각 데이터 열에 대한 메모 -> 열 이름의 의미를 설명 등

In [5]:
college = pd.read_csv(p1 / 'college.csv')
different_cols = ['RELAFFIL', 'SATMTMID', 'CURROPER', 'INSTNM', 'STABBR']
col2 = college.loc[:, different_cols]
col2.head()

,RELAFFIL,SATMTMID,CURROPER,INSTNM,STABBR
0,0,420.0,1,Alabama A & M University,AL
1,0,565.0,1,University of Alabama at Birmingham,AL
2,1,NaN,1,Amridge University,AL
3,0,590.0,1,University of Alabama in Huntsville,AL
4,0,430.0,1,Alabama State University,AL


In [7]:
# memory_usage(deep=True) 메서드 사용해 각 열의 메모리 사용량 확인
original_mem = col2.memory_usage(deep=True)
original_mem

Index          128
RELAFFIL     60280
SATMTMID     60280
CURROPER     60280
INSTNM      660240
STABBR      444565
dtype: int64

In [12]:
# nunique() 메서드를 통해 카디널리티(유일한 값의 개수) 확인
col2.nunique()
# 'RELAFFIL' 등은 0과 1로만 이루어져있으므로 astype() 메서드 활용하여 int8로 변환
col2['RELAFFIL'] = col2['RELAFFIL'].astype('int8')
col2['CURROPER'] = col2['CURROPER'].astype('int8')
# 'STABBR'의 경우 object 형식 중 카디널리티가 낮으므로 category 형식으로 변환
# groupby 활용 시 observed=True 인자 전달 필요
col2['STABBR'] = col2['STABBR'].astype('category')
col2.memory_usage(deep=True)

Index          128
RELAFFIL      7535
SATMTMID     60280
CURROPER      7535
INSTNM      660699
STABBR       13120
dtype: int64

In [11]:
col2.nunique()

RELAFFIL       2
SATMTMID     167
CURROPER       2
INSTNM      7535
STABBR        59
dtype: int64

In [13]:
movie = pd.read_csv(p1 / 'movie.csv')
movie2 = movie[['movie_title', 'imdb_score', 'budget']]
movie2.head()

,movie_title,imdb_score,budget
0,Avatar,7.9,237000000.0
1,Pirates of the Caribbean: At World's End,7.1,300000000.0
2,Spectre,6.8,245000000.0
3,The Dark Knight Rises,8.5,250000000.0
4,Star Wars: Episode VII - The Force Awakens,7.1,NaN


In [16]:
# nlargest() 메서드 활용하여 상위 n개 설정 가능
(movie2
.nlargest(100, 'imdb_score')
# nsmallest() 메서드를 체인 시켜 그 중 하위 n개 설정 가능
.nsmallest(5, 'budget')
)

,movie_title,imdb_score,budget
4804,Butterfly Girl,8.7,180000.0
4801,Children of Heaven,8.5,180000.0
4706,12 Angry Men,8.9,350000.0
4550,A Separation,8.4,500000.0
4636,The Other Dream Team,8.4,500000.0


In [22]:
# 단 100번째 imdb_score가 그 이후 순위들의 점수와 동일할 경우 보정할 필요(e.g. 8.4)
# nlargest() 메서드는 Series에서도 활용 가능(두 번째 인자 불필요)
(movie2[movie2['imdb_score'] >= movie2['imdb_score'].nlargest(100).min()]
.nsmallest(5, 'budget')
)

,movie_title,imdb_score,budget
4815,A Charlie Brown Christmas,8.4,150000.0
4801,Children of Heaven,8.5,180000.0
4804,Butterfly Girl,8.7,180000.0
4706,12 Angry Men,8.9,350000.0
4550,A Separation,8.4,500000.0


In [27]:
movie3 = movie[['movie_title', 'title_year', 'imdb_score']]
movie3.head()

,movie_title,title_year,imdb_score
0,Avatar,2009.0,7.9
1,Pirates of the Caribbean: At World's End,2007.0,7.1
2,Spectre,2015.0,6.8
3,The Dark Knight Rises,2012.0,8.5
4,Star Wars: Episode VII - The Force Awakens,NaN,7.1


In [33]:
# sort_values() 메서드 사용하여 정렬 진행 -> by 파라미터로 희망하는 열 전달, ascending 파라미터로 오름차순 여부 결정
(movie3
.sort_values(by=['title_year', 'imdb_score'], ascending=[False, False])
# drop_duplicates() 메서드 이용하여 해당하는 열의 중복 값 제외 -> subset 파라미터 활용
# keep 파라미터를 활용해 원하는 값을 남길 수 있음 : 'first', 'last', False
.drop_duplicates(subset='title_year')
)

,movie_title,title_year,imdb_score
4312,Kickboxer: Vengeance,2016.0,9.1
3745,Running Forever,2015.0,8.6
4369,Queen of the Mountains,2014.0,8.7
3935,"Batman: The Dark Knight Returns, Part 2",2013.0,8.4
3,The Dark Knight Rises,2012.0,8.5
...,...,...,...
2694,Metropolis,1927.0,8.3
4767,The Big Parade,1925.0,8.3
4833,Over the Hill to the Poorhouse,1920.0,4.8
4695,Intolerance: Love's Struggle Throughout the Ages,1916.0,8.0


In [36]:
# groupby를 이용해 복제 진행
(movie3
# groupby에서 ['imdb_score'].max()를 이용하면 모든 열들이 유지가 안됨 -> apply 활용해야
.groupby('title_year', as_index=False)
# apply에 lambda 식을 활용하여 imdb_score의 최상단 첫번째만 남기기
.apply(lambda df: df.sort_values('imdb_score', ascending=False).head(1))
# droplevel()을 통해 인덱스의 레벨 삭제
.droplevel(0)
)

,movie_title,title_year,imdb_score
4695,Intolerance: Love's Struggle Throughout the Ages,1916.0,8.0
4833,Over the Hill to the Poorhouse,1920.0,4.8
4767,The Big Parade,1925.0,8.3
2694,Metropolis,1927.0,8.3
4555,Pandora's Box,1929.0,8.0
...,...,...,...
3,The Dark Knight Rises,2012.0,8.5
3935,"Batman: The Dark Knight Returns, Part 2",2013.0,8.4
4369,Queen of the Mountains,2014.0,8.7
3745,Running Forever,2015.0,8.6


In [38]:
# sort_values() 메서드를 이용한 nlargest / nsmallest 복제
(movie2
# kind='mergsort' 인자 전달하여 nlargest와 같은 기준으로 정렬 가능
.sort_values('imdb_score', ascending=False, kind='mergsort')
.head(100)
.sort_values('budget', ascending=True)
.head(5)
)

,movie_title,imdb_score,budget
4804,Butterfly Girl,8.7,180000.0
4801,Children of Heaven,8.5,180000.0
4706,12 Angry Men,8.9,350000.0
4550,A Separation,8.4,500000.0
4636,The Other Dream Team,8.4,500000.0
